In [1]:
import pandas as pd
from collections import Counter
import ast
import json
from tqdm import tqdm

In [2]:
# Load the CSV file (replace 'your_file.csv' with your actual filename)
file_path = '../../../Data/Raw_data/AirBnB_data.csv'
df = pd.read_csv(file_path, low_memory=False)
df.shape

(953953, 26)

In [3]:
category_columns = ['Property Type', 'Listing Type', 'Airbnb Response Time (Text)', 'Cancellation Policy',
                    'Check-in Time', 'Checkout Time']  # 'Amenities' 는 특수처리
binary_columns = ['Airbnb Superhost', 'Instantbook Enabled', 'Pets Allowed']
numerical_columns = ['Bedrooms', 'Bathrooms', 'Max Guests', 'Available Days', 'Blocked Days', 
                    'Response Rate', 'Number of Reviews', 'Minimum Stay', 'Number of Photos', 'Overall Rating']
listing_title = ['Listing Title']

print(len(category_columns), len(binary_columns), len(numerical_columns))

6 3 10


In [4]:
# Amenities 전처리
def parse_amenities(amenities):
    try:
        if isinstance(amenities, str):
            if amenities.startswith("{"):  # JSON 형식
                amenities_dict = json.loads(amenities)  # JSON 파싱
                return [item.lower() for sublist in amenities_dict.values() for item in sublist]  # 소문자 변환
            elif amenities.startswith("["):  # 리스트 문자열 형식
                return [item.lower() for item in json.loads(amenities)]  # 소문자 변환
        return []
    except:
        return []

# 데이터 변환 적용
df["Amenities"] = df["Amenities"].apply(parse_amenities)

In [5]:
# Flatten & 빈도수 계산
all_amenities = [amenity for sublist in df["Amenities"] for amenity in sublist]
amenity_counts = Counter(all_amenities)
print(len(amenity_counts))

# 상위 50개 Amenities 출력
top_amenities = [amenity for amenity, count in amenity_counts.most_common(50)]
print("Top Amenities:", top_amenities)

343
Top Amenities: ['heating', 'wireless_internet', 'ac', 'essentials', 'kitchen', 'shampoo', 'washer', 'hair-dryer', 'hangers', 'fire_extinguisher', 'smoke_detector', 'laptop-friendly', 'tv', 'hot_water', 'iron', 'carbon_monoxide_detector', 'refrigerator', 'dishes_and_silverware', 'microwave', 'long_term_stays_allowed', 'first_aid_kit', 'stove', 'cable', 'bed_linens', 'cooking_basics', 'elevator', 'dryer', 'keypad', 'private-entrance', 'luggage_dropoff_allowed', 'free_parking', 'lock_on_bedroom_door', 'extra_pillows_and_blankets', 'breakfast', 'coffee_maker', 'paid_parking', 'room_darkening_shades', 'paid_parking_on_premises', 'host_checkin', 'pocket_wifi', 'ethernet_connection', 'window_guards', 'cleaning_before_checkout', 'patio_or_balcony', 'garden_or_backyard', 'street_parking', 'oven', 'internet', 'smartlock', 'single_level_home']


In [6]:
# 데이터 분리
dong_names = list(df['Dong_name'].unique())
dong_names.append('상계8동')

date_range = sorted(df['Reporting Month'].unique())
print(len(dong_names), len(date_range))

full_index = pd.MultiIndex.from_product([date_range, dong_names], names=['Reporting Month', 'Dong_name'])
df_keys = pd.DataFrame(index=full_index).reset_index()
df_keys

424 67


,Reporting Month,Dong_name
0,2017-01-01,혜화동
1,2017-01-01,사근동
2,2017-01-01,연남동
3,2017-01-01,우이동
4,2017-01-01,사직동
...,...,...
28403,2022-07-01,번3동
28404,2022-07-01,무악동
28405,2022-07-01,쌍문2동
28406,2022-07-01,시흥4동


In [7]:
for col in numerical_columns:
    count = df[col].describe()
    print(count)
    print(count['mean'])
    print('========================================')
    break
    

count    953785.000000
mean          1.263197
std           1.350749
min           0.000000
25%           1.000000
50%           1.000000
75%           1.000000
max          50.000000
Name: Bedrooms, dtype: float64
1.2631966323647363


In [8]:
df.loc[0:3, 'Listing Title']

0    [Enkor Stay] Private room H Shared Flat SKKU
1    [Enkor Stay] Private room H Shared Flat SKKU
2               Cozy studio - 3mins Hanyang Univ.
3               Cozy studio - 3mins Hanyang Univ.
Name: Listing Title, dtype: object

# Structured Summary Prompt 생성

In [9]:
SSP_df = df_keys.copy()
SSP_prompt_wo_listing = []
SSP_prompt_w_listing = []

for idx, row in tqdm(df_keys.iterrows(), total=len(df_keys), desc="Generating prompts"):
    month = row['Reporting Month']
    dong = row['Dong_name']
    
    ex_df = df.loc[(df['Reporting Month']==month) & (df['Dong_name']==dong), ]
    prompt = [f"[{month} | {dong}] AirBnB Feature Summary:"
              f"Total number of AirBnB: {len(ex_df)}"]
    
    if ex_df.empty:
        prompt.append('There is no AirBnB')
        SSP_prompt_wo_listing.append(prompt)
        SSP_prompt_w_listing.append(prompt)
        continue
    
    # 1. category_columns 처리
    category_prompt = [f"Category Column Attributes"]
    for col in category_columns:
        #print(col)
        category = [f"\nCategory: {col} Information: Total number with data: {len(ex_df[col])}"]
        vc = ex_df[col].value_counts()
        for val, count in vc.items():
            category.append(f"   {val}: {count}")
        category_prompt.extend(category)
    
    # <Amenities 처리> -> 리스트 내용 중 top_amenities에 있는 것만 추가
    amenities_series = ex_df['Amenities']
    total_amenities = []
    for val in amenities_series:
        try:
            items = ast.literal_eval(val) if isinstance(val, str) else val
            if isinstance(items, list):
                total_amenities.extend(items)  # 리스트를 펼쳐서 하나로 합쳐줌
        except:
            continue
    counts = Counter(total_amenities)
    
    # top_amenities 중에서만 필터링
    top_amenity_counts = {amenity: counts[amenity] for amenity in top_amenities if amenity in counts}
    
    category = [f"\nCategory: Amenities Information: Total number with data: {len(total_amenities)}"]
    for k, v in top_amenity_counts.items():
        category.append(f"   {k}: {v}")
    category_prompt.extend(category)
    category_prompt.append('---------------------------------------------------------------------------\n')
    
    # 2. binary_columns 처리
    binary_prompt = [f"Binary Column Attributes\n"]
    for col in binary_columns:
        #print(col)
        col_data = ex_df[col].dropna()
        total = len(col_data)
        
        # 값이 하나도 없으면 건너뛰기
        if total == 0:
            binary_prompt.append(f"{col} Information: No data available")
            continue
        
        binary = [f"{col} Information: Total number with data: {total}"]
        count = ex_df[col].sum()
        binary.append(f"number of {col}: {count}")
        binary_prompt.extend(binary)
    binary_prompt.append('---------------------------------------------------------------------------\n')
    
    # 3. numerical_columns 처리
    numerical_prompt = [f"Numerical Column Attributes"]
    for col in numerical_columns:
        #print(col)
        col_data = ex_df[col].dropna()
        total = len(col_data)
        
        # 값이 하나도 없으면 건너뛰기
        if total == 0:
            numerical_prompt.append(f"{col} Information: No data available")
            continue
        
        numerical = [f"{col} Information: Total number with data: {total}"]
        stats = col_data.describe()
        numerical.append(f"   Mean: {stats['mean']:.2f}")
        numerical.append(f"   Std Dev: {stats['std']:.2f}")
        numerical.append(f"   Median: {col_data.median():.2f}")
        numerical.append(f"   Min: {stats['min']:.2f}")
        numerical.append(f"   Max: {stats['max']:.2f}")
        numerical_prompt.extend(numerical)
    numerical_prompt.append('---------------------------------------------------------------------------\n')
        
    # 4. listing title 처리 (옵션)
    listing_series = ex_df['Listing Title']
    listing_list = []
    total = len(listing_series)
    
    listing_prompt = [f"AirBnB Listing Information: Total number with data: {total}"]
    
    for val in listing_series:
        listing_prompt.append(f"\'{val}\', ")
    
    listing_prompt.append('---------------------------------------------------------------------------\n')
    
    last_prompt = [f'Assume you are a data analyst that is familiar with AirBnB market. Give me the embedding of this {dong} at {month}']
    
    # 최종 프롬프트 합치기
    full_prompt_wo_listing = "\n".join(
        prompt + category_prompt + binary_prompt + numerical_prompt + last_prompt
    )
    full_prompt_w_listing = "\n".join(
        prompt + category_prompt + binary_prompt + numerical_prompt + listing_prompt + last_prompt
    )

    # 문자열로 저장
    SSP_prompt_wo_listing.append(full_prompt_wo_listing)
    SSP_prompt_w_listing.append(full_prompt_w_listing)
    

Generating prompts: 100%|██████████| 28408/28408 [41:52<00:00, 11.31it/s]


In [19]:
print(len(SSP_prompt_wo_listing))
print(len(SSP_prompt_w_listing))
SSP_prompt_wo_listing

28408
28408


['[2017-01-01 | 혜화동] AirBnB Feature Summary:Total number of AirBnB: 84\nCategory Column Attributes\n\nCategory: Property Type Information: Total number with data: 84\n   House: 24\n   Bed and breakfast: 18\n   Rental unit: 10\n   Home: 9\n   Apartment: 5\n   Pension: 4\n   Bed & Breakfast: 4\n   Guesthouse: 2\n   Hostel: 2\n   Room in bed and breakfast: 1\n   Entire bed & breakfast: 1\n   Home/apt: 1\n   Pension (south korea): 1\n   Shared room in house: 1\n   Private room in house: 1\n\nCategory: Listing Type Information: Total number with data: 84\n   Private room: 42\n   Entire home/apt: 28\n   Shared room: 13\n   Hotel room: 1\n\nCategory: Airbnb Response Time (Text) Information: Total number with data: 84\n   within an hour: 46\n   within a few hours: 6\n   within a day: 3\n\nCategory: Cancellation Policy Information: Total number with data: 84\n   moderate: 23\n   strict_14_with_grace_period: 15\n   flexible: 13\n   Moderate: 9\n   strict: 8\n   Strict: 4\n   Free cancellation fo

In [21]:
airbnb_SSP_wo_listing = df_keys.copy()
airbnb_SSP_w_listing = df_keys.copy()

In [22]:
airbnb_SSP_wo_listing['prompt'] = SSP_prompt_wo_listing
airbnb_SSP_w_listing['prompt'] = SSP_prompt_w_listing

In [23]:
print(airbnb_SSP_wo_listing.shape)
print(airbnb_SSP_w_listing.shape)

airbnb_SSP_wo_listing.head()

# 여기까지 실행함...

(28408, 3)
(28408, 3)


,Reporting Month,Dong_name,prompt
0,2017-01-01,혜화동,[2017-01-01 | 혜화동] AirBnB Feature Summary:Tota...
1,2017-01-01,사근동,[2017-01-01 | 사근동] AirBnB Feature Summary:Tota...
2,2017-01-01,연남동,[2017-01-01 | 연남동] AirBnB Feature Summary:Tota...
3,2017-01-01,우이동,[2017-01-01 | 우이동] AirBnB Feature Summary:Tota...
4,2017-01-01,사직동,[2017-01-01 | 사직동] AirBnB Feature Summary:Tota...


In [24]:
airbnb_SSP_w_listing.head()

,Reporting Month,Dong_name,prompt
0,2017-01-01,혜화동,[2017-01-01 | 혜화동] AirBnB Feature Summary:Tota...
1,2017-01-01,사근동,[2017-01-01 | 사근동] AirBnB Feature Summary:Tota...
2,2017-01-01,연남동,[2017-01-01 | 연남동] AirBnB Feature Summary:Tota...
3,2017-01-01,우이동,[2017-01-01 | 우이동] AirBnB Feature Summary:Tota...
4,2017-01-01,사직동,[2017-01-01 | 사직동] AirBnB Feature Summary:Tota...


In [25]:
airbnb_SSP_wo_listing.to_csv('../dong_prompts_new/AirBnB_SSP_wo_prompts.csv', index=False)
airbnb_SSP_w_listing.to_csv('../dong_prompts_new/AirBnB_SSP_w_prompts.csv', index=False)